In [1]:
import findspark

ModuleNotFoundError: No module named 'findspark'

In [2]:
findspark.init() # for initialization of SPARK context

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd


In [5]:
spark=SparkSession.builder.getOrCreate() # if there is any existing session it will not create new

### Reading it from VP IP

customer_details = spark.read.csv('hdfs://192.168.232.128:8020/user/cloudera/hackathon/customer_details.csv',header=True)
customer_cdr = spark.read.csv('hdfs://192.168.232.128:8020/user/cloudera/hackathon/customer_cdr.csv',header=True)
plan_details = spark.read.csv('hdfs://192.168.232.128:8020/user/cloudera/hackathon/plan_details.csv',header=True) 

In [6]:
# now not pandas but SPARK needs to read the files

h_cdr = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\cdr.csv", header = True)
h_complaint = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\complaint.csv", header = True)
h_cust_churn = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\customer_churn_demography.csv", header = True)
h_revenue = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\revenue.csv", header = True)
h_signal = spark.read.csv("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\signalstrength.csv", header = True)

In [7]:
h_df1 = (h_cust_churn.join(h_cdr, "phone number")
        .join(h_complaint, "phone number")
        .join(h_revenue, "phone number")
        .join(h_signal, "phone number"))

In [9]:
h_df1.show(1)

+------------+-----+-----+--------------+---------+--------------+---+------+-------------------+------------+------------------------+------------------+---------------+-----------+-------------+---------------------+-----------------+---------------+-----------------+---------------+-------------------+-----------------+------------------+----------------+----------------------+--------------------------+-------------------+-----------------------------------------------------------------------------+-------------+-------------------+----------+----------------------+-----+--------------+----------------------------+---------------------+-------------------------------------------------+----------------+----------------+------------------+-----------------+-----+-----------------+-----------------+----------------------------------+---------------------------------+
|phone number|churn|state|account length|area code|account (days)|Age|Gender|         Occupation|Joining date|Subscipti

#### Deleting a data column from the first complete dataframe

df.drop('age').collect() # connect() would keep it in the RAM for faster computation . This is not needed now

df.drop(df.age).collect()

In [11]:
# Understanding the feature datatype

h_df1.printSchema()

root
 |-- phone number: string (nullable = true)
 |-- churn: string (nullable = true)
 |-- state: string (nullable = true)
 |-- account length: string (nullable = true)
 |-- area code: string (nullable = true)
 |-- account (days): string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Joining date: string (nullable = true)
 |-- Subsciption renewal date: string (nullable = true)
 |-- international plan: string (nullable = true)
 |-- voice mail plan: string (nullable = true)
 |-- %call drops: string (nullable = true)
 |-- %Packet_drops: string (nullable = true)
 |-- number vmail messages: string (nullable = true)
 |-- total day minutes: string (nullable = true)
 |-- total day calls: string (nullable = true)
 |-- total eve minutes: string (nullable = true)
 |-- total eve calls: string (nullable = true)
 |-- total night minutes: string (nullable = true)
 |-- total night calls: string (nullable = true)

In [12]:
h_df1.count() # to check number of rows

3333

In [14]:
### Summary statistcs
h_df1.describe('churn').show()

+-------+-----+
|summary|churn|
+-------+-----+
|  count| 3333|
|   mean| null|
| stddev| null|
|    min|FALSE|
|    max| TRUE|
+-------+-----+



In [37]:


kk= h_df1.summary().repartition(1) # using summary function to include more statistics as compared to describe()

# we are doing repartition to 1 here as while checking kk.rdd.getNumPartitions() we saw there are 4 partition and needed to join the same

In [38]:
#Saving the file in local drive as CSV
kk.write.save("C:\\Users\\inkpathak\\Desktop\\Hackathon\\Data\\descriptiveStatistics1.csv", format="csv", header = True)

# This function cant overwrite existing table and hence create new table everytime

In [19]:
kk.rdd.getNumPartitions()  

1

In [24]:
#Find the number of distinct levels in selected coluns

h_df1.select('voice mail plan').distinct().count()


2

In [25]:
h_df1.select('total night calls').distinct().count()

120

### Type casting of the data for analysis

this is required as different algorithm does not work on string data and hence need to convert variables into float selectively

In [8]:
from pyspark.sql.types import *
h_final = h_df1.withColumn("account length", h_df1["account length"].cast(FloatType()))\
.withColumn("account (days)", h_df1["account (days)"].cast(FloatType()))\
.withColumn("Age", h_df1["Age"].cast(FloatType()))\
.withColumn("%call drops", h_df1["%call drops"].cast(FloatType()))\
.withColumn("%Packet_drops", h_df1["%Packet_drops"].cast(FloatType()))\
.withColumn("number vmail messages", h_df1["number vmail messages"].cast(FloatType()))\
.withColumn("total day minutes", h_df1["total day minutes"].cast(FloatType()))\
.withColumn("total day calls", h_df1["total day calls"].cast(FloatType()))\
.withColumn("total eve minutes", h_df1["total eve minutes"].cast(FloatType()))\
.withColumn("total eve calls", h_df1["total eve calls"].cast(FloatType()))\
.withColumn("total night minutes", h_df1["total night minutes"].cast(FloatType()))\
.withColumn("total night calls", h_df1["total night calls"].cast(FloatType()))\
.withColumn("total intl minutes", h_df1["total intl minutes"].cast(FloatType()))\
.withColumn("total intl calls", h_df1["total intl calls"].cast(FloatType()))\
.withColumn("customer service calls", h_df1["customer service calls"].cast(FloatType()))\
.withColumn("Number of complaint raised", h_df1["Number of complaint raised"].cast(FloatType()))\
.withColumn("Time for resolution", h_df1["Time for resolution"].cast(FloatType()))\
.withColumn("payment_made_but_order_not_placed_and_amount_is_deducted_from_my_bank_account", h_df1["payment_made_but_order_not_placed_and_amount_is_deducted_from_my_bank_account"].cast(FloatType()))\
.withColumn("Billing_issue", h_df1["Billing_issue"].cast(FloatType()))\
.withColumn("Calls_are_not_going", h_df1["Calls_are_not_going"].cast(FloatType()))\
.withColumn("drop_calls", h_df1["drop_calls"].cast(FloatType()))\
.withColumn("No_internet_connection", h_df1["No_internet_connection"].cast(FloatType()))\
.withColumn("order", h_df1["order"].cast(FloatType()))\
.withColumn("payment_refund", h_df1["payment_refund"].cast(FloatType()))\
.withColumn("recharge_offer_service_fraud", h_df1["recharge_offer_service_fraud"].cast(FloatType()))\
.withColumn("Slow_network_coverage", h_df1["Slow_network_coverage"].cast(FloatType()))\
.withColumn("uninformed___unsubscribed_weekly_amount_deduction", h_df1["uninformed___unsubscribed_weekly_amount_deduction"].cast(FloatType()))\


In [9]:
h_final.printSchema()

root
 |-- phone number: string (nullable = true)
 |-- churn: string (nullable = true)
 |-- state: string (nullable = true)
 |-- account length: float (nullable = true)
 |-- area code: string (nullable = true)
 |-- account (days): float (nullable = true)
 |-- Age: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Joining date: string (nullable = true)
 |-- Subsciption renewal date: string (nullable = true)
 |-- international plan: string (nullable = true)
 |-- voice mail plan: string (nullable = true)
 |-- %call drops: float (nullable = true)
 |-- %Packet_drops: float (nullable = true)
 |-- number vmail messages: float (nullable = true)
 |-- total day minutes: float (nullable = true)
 |-- total day calls: float (nullable = true)
 |-- total eve minutes: float (nullable = true)
 |-- total eve calls: float (nullable = true)
 |-- total night minutes: float (nullable = true)
 |-- total night calls: float (nullable = true)
 |-- total 

## Handling Missing value

We are going to look into some option to handle missing data

In [59]:
# First option to deal with missing value is to delete the rows with missing value

# before doing this step assigning the data frame to another name 

h_df = h_final

In [52]:
# What if I want to drop the all rows with null value

# cheking number of missing value in the dataframe

h_df1.count() - h_final.dropna().count()

3

In [48]:
# Function to generate number of missing value by column

def count_nulls(df):
    null_counts = []          #make an empty list to hold our results
    for col in df.dtypes:     #iterate through the column data types we saw above, e.g. ('C0', 'bigint')
        cname = col[0]        #splits out the column name, e.g. 'C0'    
        ctype = col[1]        #splits out the column type, e.g. 'bigint'
        if ctype != 'string': #skip processing string columns for efficiency (can't have nulls)
            nulls = df.where( df[cname].isNull() ).count()
            result = tuple([cname, nulls])  #new tuple, (column name, null count)
            null_counts.append(result)      #put the new tuple in our result list
    return null_counts

In [60]:
null_counts = count_nulls(h_final)

In [61]:
null_counts

[('account length', 0),
 ('account (days)', 0),
 ('Age', 0),
 ('%call drops', 0),
 ('%Packet_drops', 2),
 ('number vmail messages', 1),
 ('total day minutes', 0),
 ('total day calls', 0),
 ('total eve minutes', 0),
 ('total eve calls', 0),
 ('total night minutes', 0),
 ('total night calls', 0),
 ('total intl minutes', 0),
 ('total intl calls', 0),
 ('customer service calls', 0),
 ('Number of complaint raised', 0),
 ('Time for resolution', 0),
 ('payment_made_but_order_not_placed_and_amount_is_deducted_from_my_bank_account',
  0),
 ('Billing_issue', 0),
 ('Calls_are_not_going', 0),
 ('drop_calls', 0),
 ('No_internet_connection', 0),
 ('order', 0),
 ('payment_refund', 0),
 ('recharge_offer_service_fraud', 0),
 ('Slow_network_coverage', 0),
 ('uninformed___unsubscribed_weekly_amount_deduction', 0)]

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

In [10]:

h_df2= h_final.select('churn','%call drops', 'total day minutes')

In [6]:
h_df2.columns

['churn', '%call drops', 'total day minutes']

In [76]:
features = h_df2.corr()

In [77]:
features.

PythonRDD[1243] at RDD at PythonRDD.scala:48

In [74]:
from pyspark.mllib.stat import Statistics

r1 = Correlation.corr(h_df2, "features").head()


AnalysisException: "cannot resolve '`features`' given input columns: [number vmail messages, total day minutes, churn, %Packet_drops, %call drops];;\n'Project ['features]\n+- AnalysisBarrier\n      +- Project [churn#5892, %call drops#14353, %Packet_drops#14400, number vmail messages#14447, total day minutes#14494]\n         +- Project [phone number#5896, churn#5892, state#5893, account length#14212, area code#5895, account (days)#14259, Age#14306, Gender#5899, Occupation#5900, Joining date#5901, Subsciption renewal date#5902, international plan#5816, voice mail plan#5817, %call drops#14353, %Packet_drops#14400, number vmail messages#14447, total day minutes#14494, total day calls#14541, total eve minutes#14588, total eve calls#14635, total night minutes#14682, total night calls#14729, total intl minutes#14776, total intl calls#14823, ... 22 more fields]\n            +- Project [phone number#5896, churn#5892, state#5893, account length#14212, area code#5895, account (days)#14259, Age#14

In [7]:
jj=h_df2.corr("%call drops","total day minutes")

In [8]:
jj

0.11816459600567038

In [ ]:
from pyspark.mllib.stat import Statistics
import pandas as pd
import numpy as np
# result can be used w/ seaborn's heatmap
def compute_correlation_matrix(df, method='pearson'):
    # wrapper around
    # https://forums.databricks.com/questions/3092/how-to-calculate-correlation-matrix-with-all-colum.html
    df_rdd = df.rdd.map(lambda row: row[0:])
    corr_mat = Statistics.corr(df_rdd, method=method)
    corr_mat_df = pd.DataFrame(corr_mat,
                    columns=df.columns, 
                    index=df.columns)
    return corr_mat_df

In [13]:
jjj=compute_correlation_matrix(df=h_df2)

AttributeError: 'SparkSession' object has no attribute 'Statistics'

In [14]:
type(h_df2)

pyspark.sql.dataframe.DataFrame

In [101]:

def compute_correlation_matrix(df, method='pearson'):
    # wrapper around
    # https://forums.databricks.com/questions/3092/how-to-calculate-correlation-matrix-with-all-colum.html
    df_rdd = df.rdd.map(lambda row: row[0:])
    corr_mat = Statistics.corr(df_rdd, method=method)
    return corr_mat

In [102]:
jjj=compute_correlation_matrix(df=h_df2, method='pearson')

Py4JJavaError: An error occurred while calling o1561.corr.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 471.0 failed 1 times, most recent failure: Lost task 0.0 in stage 471.0 (TID 3014, localhost, executor driver): org.apache.spark.SparkException: Python worker did not connect back in time
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:143)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:72)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:86)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

In [112]:
features = h_df2.rdd.map(lambda row: row[0:])

from pyspark.mllib.stat import Statistics

corr_mat=Statistics.corr(features, method="pearson")

Py4JJavaError: An error occurred while calling o1746.corr.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 482.0 failed 1 times, most recent failure: Lost task 0.0 in stage 482.0 (TID 3025, localhost, executor driver): org.apache.spark.SparkException: Python worker did not connect back in time
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:143)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:72)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:86)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

In [17]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
spark = SparkSession.builder.appName('correlation').getOrCreate()
data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

Py4JJavaError: An error occurred while calling z:org.apache.spark.ml.stat.Correlation.corr.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 16, localhost, executor driver): org.apache.spark.SparkException: Python worker did not connect back in time
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:143)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:72)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:86)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.r

In [16]:
data_train_rdd=h_df2.rdd

In [22]:
v1=data_train_rdd.map(lambda x:x[1])

In [23]:
v2=data_train_rdd.map(lambda x:x[2])

In [20]:
from pyspark.mllib.stat import Statistics
NewVector=data_train_rdd.map(lambda x:x[1:2]) 


# correlation_matrix = Statistics.corr(NewVector, method="pearson")